### Inventory Breakdown Data Analysis
* Elemental Inventory Report- Shares several fields with inventory by Subinventory report but has the values for Material, Overhead, etc.. These fields need to be "stacked".  

* Inventory by Subinventory-  Brings in Frozen "Std" Cost and whether part is a Make or Buy item.

* Accounts- Provides GL Acct numbers for the value fields Material, Overhead, etc.
-----

### Note
* Instructions have been included for each segment. It is critical to upload reports exactly as described in the "Data.xlsx/Instructions" or Python will not recognize and import file.

* Data Notes worksheet in the Data.xlsx workbook provides some explanation of the dataframe outputs including troubleshooting.


In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from babel.numbers import format_currency
!pip install XlsxWriter
import xlsxwriter

# File should be imported into Colab and sitting in left pane in the File icon.

#Importing the required file
data=pd.ExcelFile('Data.xlsx')

# Read Excel worksheets
elemental_inv_data=pd.read_excel(data,'Elemental_Inventory')
inv_bySubinv_data=pd.read_excel(data,'Inv_bySubinv')
accounts=pd.read_excel(data,'Accounts')

# Store into Pandas data frame
elemental_inv_df=pd.DataFrame(elemental_inv_data)
inv_bySubinv_df=pd.DataFrame(inv_bySubinv_data)
accounts_df=pd.DataFrame(accounts)

## View Files in their dataframes



### Elemental Inventory dataframe

In [2]:
# View dataframe.  Verify number of rows looks correct and values have been imported.

elemental_inv_df

,Part Number,Description,Category,Subinventory,UOM,Quantity,Material,Resource,Overhead,Mat'l OH,Out Proc,Total
0,1752699,"Flat Coil 0.1793"" Nom / 59.0"" / MS252 /",PUR-RAW,FAB,LB,1,2,0,0,3,0,6
1,1764529,"Flat Coil 0.1793"" Nom / 52.0"" / MS264",PUR-RAW,COIL 8,LB,1,2,0,0,3,0,6
2,1764529,"Flat Coil 0.1793"" Nom / 52.0"" / MS264",PUR-RAW,DECOIL,LB,1,2,0,0,3,0,6
3,1764699,"Flat Coil 0.1793"" Nom / 59.0"" / MS264",PUR-RAW,COIL 33,LB,1,2,0,0,3,0,6
4,1764699,"Flat Coil 0.1793"" Nom / 59.0"" / MS264",PUR-RAW,DECOIL,LB,1,2,0,0,3,0,6
5,1764699,"Flat Coil 0.1793"" Nom / 59.0"" / MS264",PUR-RAW,FAB 1,LB,1,2,0,0,3,0,6
6,5164555,"Flat Coil 0.3125"" Nom / 53.5"" / MS264",PUR-RAW,FAB,LB,1,2,0,0,3,0,6
7,5164699,"Flat Coil 0.3125"" Nom / 59.0"" / MS264",PUR-RAW,COIL 12,LB,1,2,0,0,3,0,6
8,5164699,"Flat Coil 0.3125"" Nom / 59.0"" / MS264",PUR-RAW,DECOIL 1,LB,1,2,0,0,3,0,6
9,5164699,"Flat Coil 0.3125"" Nom / 59.0"" / MS264",PUR-RAW,FAB,LB,1,2,0,0,3,0,6


In [3]:
# Drop Total columns
elemental_inv_df=elemental_inv_df.drop(columns=["Total"])

# Sort by Part Number. Ascending is default sort option. 
elemental_inv_df.sort_values(by=['Part Number','Quantity'],inplace=True,ignore_index=True)

# Change Part Number to string data type "object".
elemental_inv_df['Part Number'] = elemental_inv_df['Part Number'].astype(str)

# Change Quantity to values data type
elemental_inv_df['Quantity'] = elemental_inv_df['Quantity'].astype(float)

# Round numerical values to three decimal places.  Preferred method over df.round()
elemental_inv_df['Quantity'] = np.round(elemental_inv_df['Quantity'], decimals = 3)
elemental_inv_df['Material'] = np.round(elemental_inv_df['Material'], decimals = 3)
elemental_inv_df['Resource'] = np.round(elemental_inv_df['Resource'], decimals = 3)
elemental_inv_df['Overhead'] = np.round(elemental_inv_df['Overhead'], decimals = 3)
elemental_inv_df["Mat'l OH"] = np.round(elemental_inv_df["Mat'l OH"], decimals = 3)
elemental_inv_df['Out Proc'] = np.round(elemental_inv_df['Out Proc'], decimals = 3)

# Check data types
elemental_inv_df.dtypes

Part Number      object
Description      object
Category         object
Subinventory     object
UOM              object
Quantity        float64
Material          int64
Resource          int64
Overhead          int64
Mat'l OH          int64
Out Proc          int64
dtype: object

In [48]:
# View dataframe.  Verify number of rows looks correct and values have not been corrupted.

# Strip leading and trailing spaces from fields to avoid concat and merge issues later.
elemental_inv_df["Subinventory"] = elemental_inv_df["Subinventory"].str.strip()
elemental_inv_df["Part Number"] = elemental_inv_df["Part Number"].str.strip()
elemental_inv_df["Description"] = elemental_inv_df["Description"].str.strip()
elemental_inv_df["Part Number"] = elemental_inv_df["Part Number"].str.strip()
elemental_inv_df["UOM"] = elemental_inv_df["UOM"].str.strip()

elemental_inv_df

,Part Number,Description,Category,Subinventory,UOM,Quantity,Material,Resource,Overhead,Mat'l OH,Out Proc
0,1752699,"Flat Coil 0.1793"" Nom / 59.0"" / MS252 /",PUR-RAW,FAB,LB,1.0,2,0,0,3,0
1,1764529,"Flat Coil 0.1793"" Nom / 52.0"" / MS264",PUR-RAW,COIL 8,LB,1.0,2,0,0,3,0
2,1764529,"Flat Coil 0.1793"" Nom / 52.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,2,0,0,3,0
3,1764699,"Flat Coil 0.1793"" Nom / 59.0"" / MS264",PUR-RAW,COIL 33,LB,1.0,2,0,0,3,0
4,1764699,"Flat Coil 0.1793"" Nom / 59.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,2,0,0,3,0
5,1764699,"Flat Coil 0.1793"" Nom / 59.0"" / MS264",PUR-RAW,FAB 1,LB,1.0,2,0,0,3,0
6,5164555,"Flat Coil 0.3125"" Nom / 53.5"" / MS264",PUR-RAW,FAB,LB,1.0,2,0,0,3,0
7,5164689,"Flat Coil 0.3125"" Nom / 68.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,2,0,0,3,0
8,5164699,"Flat Coil 0.3125"" Nom / 59.0"" / MS264",PUR-RAW,COIL 12,LB,1.0,2,0,0,3,0
9,5164699,"Flat Coil 0.3125"" Nom / 59.0"" / MS264",PUR-RAW,DECOIL 1,LB,1.0,2,0,0,3,0


In [5]:
# Correct Subinventory Acct names to match Oracle list
# COIL 1, as example, does not exist in Oracle Subinventory list.  Should be "COIL"

#elemental_inv_df.replace({'Subinventory': 'COIL 8'}, {'Subinventory': 'COIL'}, regex=True, inplace = True)
#elemental_inv_df = elemental_inv_df.replace('FAB 1', 'FAB')
#elemental_inv_df = elemental_inv_df.replace(regex={'COIL 33': 'COIL'})
#elemental_inv_df = elemental_inv_df.replace(['DECOIL 1'], 'DECOIL') 
#elemental_inv_df = elemental_inv_df.replace(to_replace='COIL 33', value='COIL', inplace=True)
elemental_inv_df.replace(['COIL 33', 'COIL 8', 'COIL 1'], 'COIL', inplace=True)


elemental_inv_df


,Part Number,Description,Category,Subinventory,UOM,Quantity,Material,Resource,Overhead,Mat'l OH,Out Proc
0,1752699,"Flat Coil 0.1793"" Nom / 59.0"" / MS252 /",PUR-RAW,FAB,LB,1.0,2,0,0,3,0
1,1764529,"Flat Coil 0.1793"" Nom / 52.0"" / MS264",PUR-RAW,COIL 8,LB,1.0,2,0,0,3,0
2,1764529,"Flat Coil 0.1793"" Nom / 52.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,2,0,0,3,0
3,1764699,"Flat Coil 0.1793"" Nom / 59.0"" / MS264",PUR-RAW,COIL 33,LB,1.0,2,0,0,3,0
4,1764699,"Flat Coil 0.1793"" Nom / 59.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,2,0,0,3,0
5,1764699,"Flat Coil 0.1793"" Nom / 59.0"" / MS264",PUR-RAW,FAB 1,LB,1.0,2,0,0,3,0
6,5164555,"Flat Coil 0.3125"" Nom / 53.5"" / MS264",PUR-RAW,FAB,LB,1.0,2,0,0,3,0
7,5164689,"Flat Coil 0.3125"" Nom / 68.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,2,0,0,3,0
8,5164699,"Flat Coil 0.3125"" Nom / 59.0"" / MS264",PUR-RAW,COIL 12,LB,1.0,2,0,0,3,0
9,5164699,"Flat Coil 0.3125"" Nom / 59.0"" / MS264",PUR-RAW,DECOIL 1,LB,1.0,2,0,0,3,0


###Inventory by Subinventory dataframe



In [7]:
# View dataframe.  Verify number of rows looks correct and values have been imported.
inv_bySubinv_df

,GL Account,Subinventory,Subinventory Description,Part Number,Part Description,Cost Category,UOM,Quantity,Extended Cost,M/B,Item Status,Std Cost
0,(Null),FAB,Fab,1752699,Flat Coil 0.17,PUR-RAW,LB,1,6,Buy,Active,0.1
1,(Null),COIL,Coil,1764529,Flat Coil 0.17,PUR-RAW,LB,1,6,Buy,Active,0.1
2,(Null),DECOIL,Decoil - Steel Coils,1764529,Flat Coil 0.17,PUR-RAW,LB,1,6,Buy,Active,0.1
3,(Null),COIL,Coil,1764699,Flat Coil 0.17,PUR-RAW,LB,1,6,Buy,Active,0.1
4,(Null),DECOIL,Decoil - Steel Coils,1764699,Flat Coil 0.17,PUR-RAW,LB,1,6,Buy,Active,0.1
5,(Null),FAB,Fab,1764699,Flat Coil 0.17,PUR-RAW,LB,1,6,Buy,Active,0.1
6,(Null),FAB,Fab,5164555,Flat Coil 0.31,PUR-RAW,LB,1,6,Buy,Active,0.1
7,(Null),COIL,Coil,5164699,Flat Coil 0.31,PUR-RAW,LB,1,6,Buy,Active,0.1
8,(Null),DECOIL,Decoil - Steel Coils,5164699,Flat Coil 0.31,PUR-RAW,LB,1,6,Buy,Active,0.1
9,(Null),FAB,Fab,5164699,Flat Coil 0.31,PUR-RAW,LB,1,6,Buy,Active,0.1


In [8]:
# Drop GL Acct, Part Description, Cost category, Extended Cost, and UOM columns
invBysubinventory_df=inv_bySubinv_df.drop(columns=["GL Account","Part Description","UOM","Cost Category","Extended Cost","Quantity"])

# Sort by Part Number. Ascending is default sort option. 
invBysubinventory_df.sort_values(by=['Part Number'],inplace=True,ignore_index=True)

# Change Part Number to string data type.
invBysubinventory_df['Part Number'] = invBysubinventory_df['Part Number'].astype(str)

# Round numerical values to three decimal places.  Preferred method over df.round()
invBysubinventory_df["Std Cost"] = np.round(invBysubinventory_df["Std Cost"], decimals = 3)
#invBysubinventory_df['Quantity'] = invBysubinventory_df['Quantity'].astype(float)

# Check data types
invBysubinventory_df.dtypes


Subinventory                 object
Subinventory Description     object
Part Number                  object
M/B                          object
Item Status                  object
Std Cost                    float64
dtype: object

In [9]:
# View dataframe.  Verify number of rows looks correct and values have not been corrupted.

# Strip leading and trailing spaces from fields to avoid concat and merge issues later.
invBysubinventory_df["Subinventory"] = invBysubinventory_df["Subinventory"].str.strip()
invBysubinventory_df["Subinventory Description"] = invBysubinventory_df["Subinventory Description"].str.strip()
invBysubinventory_df["Part Number"] = invBysubinventory_df["Part Number"].str.strip()
invBysubinventory_df["M/B"] = invBysubinventory_df["M/B"].str.strip()
invBysubinventory_df["Item Status"] = invBysubinventory_df["Item Status"].str.strip()

invBysubinventory_df

,Subinventory,Subinventory Description,Part Number,M/B,Item Status,Std Cost
0,FAB,Fab,1752699,Buy,Active,0.1
1,COIL,Coil,1764529,Buy,Active,0.1
2,DECOIL,Decoil - Steel Coils,1764529,Buy,Active,0.1
3,COIL,Coil,1764699,Buy,Active,0.1
4,DECOIL,Decoil - Steel Coils,1764699,Buy,Active,0.1
5,FAB,Fab,1764699,Buy,Active,0.1
6,FAB,Fab,5164555,Buy,Active,0.1
7,DECOIL,Decoil - Steel Coils,5164689,Buy,Active,0.1
8,COIL,Coil,5164699,Buy,Active,0.1
9,DECOIL,Decoil - Steel Coils,5164699,Buy,Active,0.1


###Accounts dataframe
*This file should not change month to month

In [10]:
# View dataframe.  Verify number of rows looks correct and values have been imported.

# Change Account to int data type.
#accounts_df['Account'] = accounts_df['Account'].astype(int)

# Change Part Number to string data type.
accounts_df['Account'] = accounts_df['Account'].astype(str)

# Strip leading and trailing spaces from fields to avoid concat and merge issues later.
accounts_df["Subinventory"] = accounts_df["Subinventory"].str.strip()
accounts_df["Parameter"] = accounts_df["Parameter"].str.strip()
accounts_df["Account"] = accounts_df["Account"].str.strip()

# Check data types
accounts_df.dtypes

Subinventory    object
Parameter       object
Account         object
dtype: object

In [11]:
# View dataframe.  Verify number of rows looks correct and values have been imported.

accounts_df

,Subinventory,Parameter,Account
0,ACCELCNTR,Material,132101
1,ACCELCNTR,Resource,132303
2,ACCELCNTR,Overhead,132306
3,ACCELCNTR,Matl_OH,132302
4,ACCELCNTR,OSP,132310
...,...,...,...
295,RUAN-RCVD,Material,132101
296,RUAN-RCVD,Resource,132303
297,RUAN-RCVD,Overhead,132306
298,RUAN-RCVD,Matl_OH,132302


### Find new Subinventory Accounts in the Elemental Inventory report/dataframe that needs to be added to Accounts dataframe.

In [12]:
# Create subset of Elemental Inventory dataframe that lists unique values of Subinventory.

# Drop GL Acct, Part Description, Cost category, Extended Cost, and UOM columns
elem_inv_culled_df=elemental_inv_df.drop(columns=["Part Number","Description","UOM","Category","Material","Resource","Mat'l OH","Out Proc","Overhead","Quantity"])

# Change Subinventory to string data type.
elem_inv_culled_df["Subinventory"] = elem_inv_culled_df["Subinventory"].astype(str)

# Check data types
#elem_inv_culled_df.dtypes

# Strip leading and trailing spaces from fields to avoid concat and merge issues later.
elem_inv_culled_df["Subinventory"] = elem_inv_culled_df["Subinventory"].str.strip()

elem_inv_subs_df= elem_inv_culled_df.drop_duplicates()

# Change Subinventory to string data type.
elem_inv_subs_df["Subinventory"] = elem_inv_subs_df["Subinventory"].astype(str)

# View partial dataframe. If more than seven records are desired the (7) can be changed to reflect desired number of records. Having empty parenthesis will return all records.
#elem_inv_subs_df.head()
elem_inv_subs_df.head(15)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Subinventory
0,FAB
1,COIL 8
2,DECOIL
3,COIL 33
5,FAB 1
8,COIL 12
9,DECOIL 1
11,COIL 27
14,COIL 7
18,RUAN-RCVD


In [13]:
# Create subset of Accounts dataframe that lists unique values of Subinventory.

# Drop GL Acct, Part Description, Cost category, Extended Cost, and UOM columns
accounts_culled_df=accounts_df.drop(columns=["Account","Parameter"])

# Change Subinventory to string data type.
accounts_culled_df["Subinventory"] = accounts_culled_df["Subinventory"].astype(str)

# Check data types
#accounts_culled_df.dtypes

# Strip leading and trailing spaces from fields to avoid concat and merge issues later.
accounts_culled_df["Subinventory"] = accounts_culled_df["Subinventory"].str.strip()

accounts_culled_df2= accounts_culled_df.drop_duplicates()

accounts_culled_df2.head(7)

,Subinventory
0,ACCELCNTR
5,BISNPCA
10,BISOUTPRCS
15,BIS-QCII
20,BISSPCA
25,BISWHOLGDS
30,BOMISSUE


In [14]:
# Check for new subinventory accounts in the Elemental Inventory data that need to be added to the Accounts table. Left join.

new_accounts_df= elem_inv_subs_df.merge(accounts_culled_df2.drop_duplicates(), on=['Subinventory'], 
                   how='left', indicator=True)

#new_accounts_df

new_accounts_df.query('_merge != "both"')

,Subinventory,_merge
1,COIL 8,left_only
3,COIL 33,left_only
4,FAB 1,left_only
5,COIL 12,left_only
6,DECOIL 1,left_only
7,COIL 27,left_only
8,COIL 7,left_only


##Create Dataframes that separate Parameters (Material, Overhead, Material OH, OSP, and Resource) into new tables that can then be appended to one another.

In [15]:


# Create a DataFrame for Material values of Elemental Inventory Report
material_df = elemental_inv_df[["Part Number","Description","Category","Subinventory","UOM","Quantity","Material"]]

# Add a column with the Parameter type
material_df['Parameter']='Material'

# Rename "Material" column to Parameter for later appending to other value type tables.
material_df2 = material_df.rename(columns={'Material': 'Value','Category':'Cost Category'})

# Strip leading and trailing spaces from column values
material_df["Parameter"] = material_df["Parameter"].str.strip()

# Add Material Acct number from accounts.df based on subinventory and parameter.  Inner join.
material_df3= pd.merge(material_df2, accounts_df, how= "inner", on=['Subinventory','Parameter'])

# Change Part Number to string data type.
material_df3['Part Number'] = material_df3['Part Number'].astype(str)

# Change Quantity to numerics with decimals data type
material_df3['Quantity'] = material_df3['Quantity'].astype(float)

# Round numerical values to three decimal places.  Preferred method over df.round()
material_df3['Quantity'] = np.round(material_df3['Quantity'], decimals = 3)
material_df3['Value'] = np.round(material_df3['Value'], decimals = 3)

# Sort by Part Number. Ascending is default sort option. 
material_df3.sort_values(by=['Part Number','Quantity'],inplace=True,ignore_index=True)

# Display the material table
material_df3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Part Number,Description,Cost Category,Subinventory,UOM,Quantity,Value,Parameter,Account
0,1752699,"Flat Coil 0.1793"" Nom / 59.0"" / MS252 /",PUR-RAW,FAB,LB,1.0,2,Material,132101
1,1764529,"Flat Coil 0.1793"" Nom / 52.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,2,Material,132101
2,1764699,"Flat Coil 0.1793"" Nom / 59.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,2,Material,132101
3,19524875,"Flat Coil 0.1046"" Nom / 4.875"" / MS264 /",PUR-RAW,FAB,LB,1.0,2,Material,132101
4,22526259,"Flat Coil 0.2242"" Nom / 6.250"" / MS252 /",PUR-RAW,RUAN-RCVD,LB,1.0,2,Material,132101
5,22526259,"Flat Coil 0.2242"" Nom / 6.250"" / MS252 /",PUR-RAW,RUAN-RCVD,LB,1.0,2,Material,132101
6,22645759,"Flat Coil 0.2242"" Nom / 3.750"" / MS264 /",PUR-RAW,FAB,LB,1.0,2,Material,132101
7,22647999,"Flat Coil 0.2242"" Nom / 7.000"" / MS264 /",PUR-RAW,FAB,LB,1.0,2,Material,132101
8,22648699,"Flat Coil 0.2242"" Nom / 8.690"" / MS264 /",PUR-RAW,FAB,LB,1.0,2,Material,132101
9,25524259,"Flat Coil 0.2500"" Nom / 4.250"" / MS252 /",PUR-RAW,FAB,LB,1.0,2,Material,132101


In [16]:
material_df3.dtypes

Part Number       object
Description       object
Cost Category     object
Subinventory      object
UOM               object
Quantity         float64
Value              int64
Parameter         object
Account           object
dtype: object

In [17]:
# Create a DataFrame for Resource values of Elemental Inventory Report
resource_df = elemental_inv_df[["Part Number","Description","Category","Subinventory","UOM","Quantity","Resource"]]

# Add a column with the Parameter type
resource_df['Parameter']='Resource'

# Rename "Resource" column to Parameter for later appending to other value type tables.
resource_df2 = resource_df.rename(columns={'Resource': 'Value','Category':'Cost Category'})

# Add Resource Acct number from accounts.df based on subinventory and parameter.  Inner join.
resource_df3= pd.merge(resource_df2, accounts_df, how= "inner", on=['Subinventory','Parameter'])

# Round numerical values to three decimal places.  Preferred method over df.round()
resource_df3['Quantity'] = np.round(resource_df3['Quantity'], decimals = 3)
resource_df3['Value'] = np.round(resource_df3['Value'], decimals = 3)

# Display the resource table
resource_df3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Part Number,Description,Cost Category,Subinventory,UOM,Quantity,Value,Parameter,Account
0,1752699,"Flat Coil 0.1793"" Nom / 59.0"" / MS252 /",PUR-RAW,FAB,LB,1.0,0,Resource,132303
1,5164555,"Flat Coil 0.3125"" Nom / 53.5"" / MS264",PUR-RAW,FAB,LB,1.0,0,Resource,132303
2,5164699,"Flat Coil 0.3125"" Nom / 59.0"" / MS264",PUR-RAW,FAB,LB,1.0,0,Resource,132303
3,5764699,"Flat Coil 0.3750"" Nom / 59.0"" / MS264",PUR-RAW,FAB,LB,1.0,0,Resource,132303
4,5964699,"Flat Coil 0.5000"" Nom / 59.0"" / MS264",PUR-RAW,FAB,LB,1.0,0,Resource,132303
5,19524875,"Flat Coil 0.1046"" Nom / 4.875"" / MS264 /",PUR-RAW,FAB,LB,1.0,0,Resource,132303
6,22645759,"Flat Coil 0.2242"" Nom / 3.750"" / MS264 /",PUR-RAW,FAB,LB,1.0,0,Resource,132303
7,22647999,"Flat Coil 0.2242"" Nom / 7.000"" / MS264 /",PUR-RAW,FAB,LB,1.0,0,Resource,132303
8,22648699,"Flat Coil 0.2242"" Nom / 8.690"" / MS264 /",PUR-RAW,FAB,LB,1.0,0,Resource,132303
9,25524259,"Flat Coil 0.2500"" Nom / 4.250"" / MS252 /",PUR-RAW,FAB,LB,1.0,0,Resource,132303


In [18]:
# Create a DataFrame for Material OH values of Elemental Inventory Report
matlOH_df = elemental_inv_df[["Part Number","Description","Category","Subinventory","UOM","Quantity","Mat'l OH"]]

# Add a column with the Parameter type
matlOH_df['Parameter']='Matl_OH'

# Rename "Mat'l OH" column to Parameter for later appending to other value type tables.
matlOH_df2 = matlOH_df.rename(columns={"Mat'l OH": 'Value','Category':'Cost Category'})

# Add Material OH Acct number from accounts.df based on subinventory and parameter.  Inner join.
matlOH_df3= pd.merge(matlOH_df2, accounts_df, how= "inner", on=['Subinventory','Parameter'])

# Round numerical values to three decimal places.  Preferred method over df.round()
matlOH_df3['Quantity'] = np.round(matlOH_df3['Quantity'], decimals = 3)
matlOH_df3['Value'] = np.round(matlOH_df3['Value'], decimals = 3)

# Display the Material Overhead table
matlOH_df3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Part Number,Description,Cost Category,Subinventory,UOM,Quantity,Value,Parameter,Account
0,1752699,"Flat Coil 0.1793"" Nom / 59.0"" / MS252 /",PUR-RAW,FAB,LB,1.0,3,Matl_OH,132302
1,5164555,"Flat Coil 0.3125"" Nom / 53.5"" / MS264",PUR-RAW,FAB,LB,1.0,3,Matl_OH,132302
2,5164699,"Flat Coil 0.3125"" Nom / 59.0"" / MS264",PUR-RAW,FAB,LB,1.0,3,Matl_OH,132302
3,5764699,"Flat Coil 0.3750"" Nom / 59.0"" / MS264",PUR-RAW,FAB,LB,1.0,3,Matl_OH,132302
4,5964699,"Flat Coil 0.5000"" Nom / 59.0"" / MS264",PUR-RAW,FAB,LB,1.0,3,Matl_OH,132302
5,19524875,"Flat Coil 0.1046"" Nom / 4.875"" / MS264 /",PUR-RAW,FAB,LB,1.0,3,Matl_OH,132302
6,22645759,"Flat Coil 0.2242"" Nom / 3.750"" / MS264 /",PUR-RAW,FAB,LB,1.0,3,Matl_OH,132302
7,22647999,"Flat Coil 0.2242"" Nom / 7.000"" / MS264 /",PUR-RAW,FAB,LB,1.0,3,Matl_OH,132302
8,22648699,"Flat Coil 0.2242"" Nom / 8.690"" / MS264 /",PUR-RAW,FAB,LB,1.0,3,Matl_OH,132302
9,25524259,"Flat Coil 0.2500"" Nom / 4.250"" / MS252 /",PUR-RAW,FAB,LB,1.0,3,Matl_OH,132302


In [19]:
# Create a DataFrame for Material OH values of Elemental Inventory Report
overhead_df = elemental_inv_df[["Part Number","Description","Category","Subinventory","UOM","Quantity","Overhead"]]

# Add a column with the Parameter type
overhead_df['Parameter']='Overhead'

# Rename "Overhead" column to Parameter for later appending to other value type tables.
overhead_df2 = overhead_df.rename(columns={"Overhead": 'Value','Category':'Cost Category'})

# Add Overhead Acct number from accounts.df based on subinventory and parameter.  Inner join.
overhead_df3= pd.merge(overhead_df2, accounts_df, how= "inner", on=['Subinventory','Parameter'])

# Round numerical values to three decimal places.  Preferred method over df.round()
overhead_df3['Quantity'] = np.round(overhead_df3['Quantity'], decimals = 3)
overhead_df3['Value'] = np.round(overhead_df3['Value'], decimals = 3)

# Display the Overhead table
overhead_df3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Part Number,Description,Cost Category,Subinventory,UOM,Quantity,Value,Parameter,Account
0,1752699,"Flat Coil 0.1793"" Nom / 59.0"" / MS252 /",PUR-RAW,FAB,LB,1.0,0,Overhead,132306
1,5164555,"Flat Coil 0.3125"" Nom / 53.5"" / MS264",PUR-RAW,FAB,LB,1.0,0,Overhead,132306
2,5164699,"Flat Coil 0.3125"" Nom / 59.0"" / MS264",PUR-RAW,FAB,LB,1.0,0,Overhead,132306
3,5764699,"Flat Coil 0.3750"" Nom / 59.0"" / MS264",PUR-RAW,FAB,LB,1.0,0,Overhead,132306
4,5964699,"Flat Coil 0.5000"" Nom / 59.0"" / MS264",PUR-RAW,FAB,LB,1.0,0,Overhead,132306
5,19524875,"Flat Coil 0.1046"" Nom / 4.875"" / MS264 /",PUR-RAW,FAB,LB,1.0,0,Overhead,132306
6,22645759,"Flat Coil 0.2242"" Nom / 3.750"" / MS264 /",PUR-RAW,FAB,LB,1.0,0,Overhead,132306
7,22647999,"Flat Coil 0.2242"" Nom / 7.000"" / MS264 /",PUR-RAW,FAB,LB,1.0,0,Overhead,132306
8,22648699,"Flat Coil 0.2242"" Nom / 8.690"" / MS264 /",PUR-RAW,FAB,LB,1.0,0,Overhead,132306
9,25524259,"Flat Coil 0.2500"" Nom / 4.250"" / MS252 /",PUR-RAW,FAB,LB,1.0,0,Overhead,132306


In [20]:
# Create a DataFrame for Outside Processing (OSP) values of Elemental Inventory Report
osp_df = elemental_inv_df[["Part Number","Description","Category","Subinventory","UOM","Quantity","Out Proc"]]

# Add a column with the Parameter type
osp_df['Parameter']='OSP'

# Rename "Out Proc" column to Parameter for later appending to other value type tables.
osp_df2 = osp_df.rename(columns={"Out Proc": 'Value','Category':'Cost Category'})

# Add OSP Acct number from accounts.df based on subinventory and parameter.  Inner join.
osp_df3= pd.merge(osp_df2, accounts_df, how= "inner", on=['Subinventory','Parameter'])

# Round numerical values to three decimal places.  Preferred method over df.round()
osp_df3['Quantity'] = np.round(osp_df3['Quantity'], decimals = 3)
osp_df3['Value'] = np.round(osp_df3['Value'], decimals = 3)

osp_df3.round(decimals = 2)

# Display the OSP table
osp_df3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Part Number,Description,Cost Category,Subinventory,UOM,Quantity,Value,Parameter,Account
0,1752699,"Flat Coil 0.1793"" Nom / 59.0"" / MS252 /",PUR-RAW,FAB,LB,1.0,0,OSP,132310
1,5164555,"Flat Coil 0.3125"" Nom / 53.5"" / MS264",PUR-RAW,FAB,LB,1.0,0,OSP,132310
2,5164699,"Flat Coil 0.3125"" Nom / 59.0"" / MS264",PUR-RAW,FAB,LB,1.0,0,OSP,132310
3,5764699,"Flat Coil 0.3750"" Nom / 59.0"" / MS264",PUR-RAW,FAB,LB,1.0,0,OSP,132310
4,5964699,"Flat Coil 0.5000"" Nom / 59.0"" / MS264",PUR-RAW,FAB,LB,1.0,0,OSP,132310
5,19524875,"Flat Coil 0.1046"" Nom / 4.875"" / MS264 /",PUR-RAW,FAB,LB,1.0,0,OSP,132310
6,22645759,"Flat Coil 0.2242"" Nom / 3.750"" / MS264 /",PUR-RAW,FAB,LB,1.0,0,OSP,132310
7,22647999,"Flat Coil 0.2242"" Nom / 7.000"" / MS264 /",PUR-RAW,FAB,LB,1.0,0,OSP,132310
8,22648699,"Flat Coil 0.2242"" Nom / 8.690"" / MS264 /",PUR-RAW,FAB,LB,1.0,0,OSP,132310
9,25524259,"Flat Coil 0.2500"" Nom / 4.250"" / MS252 /",PUR-RAW,FAB,LB,1.0,0,OSP,132310


### Concatenate the elemental inventory dataframes by resource type into a single "stacked" dataframe and then...
### Merge the new elemental inventory dataframe with the Inventory by Subinventory dataframe.

In [21]:
# Concatenate the Resource dataframes into one Inventory dataframe for eventual export

Inventory_df=pd.concat([material_df3,resource_df3,matlOH_df3,overhead_df3,osp_df3],axis=0)

Inventory_df


,Part Number,Description,Cost Category,Subinventory,UOM,Quantity,Value,Parameter,Account
0,1752699,"Flat Coil 0.1793"" Nom / 59.0"" / MS252 /",PUR-RAW,FAB,LB,1.0,2,Material,132101
1,1764529,"Flat Coil 0.1793"" Nom / 52.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,2,Material,132101
2,1764699,"Flat Coil 0.1793"" Nom / 59.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,2,Material,132101
3,19524875,"Flat Coil 0.1046"" Nom / 4.875"" / MS264 /",PUR-RAW,FAB,LB,1.0,2,Material,132101
4,22526259,"Flat Coil 0.2242"" Nom / 6.250"" / MS252 /",PUR-RAW,RUAN-RCVD,LB,1.0,2,Material,132101
...,...,...,...,...,...,...,...,...,...
18,5164689,"Flat Coil 0.3125"" Nom / 68.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,0,OSP,132310
19,5764699,"Flat Coil 0.3750"" Nom / 59.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,0,OSP,132310
20,5964699,"Flat Coil 0.5000"" Nom / 59.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,0,OSP,132310
21,22526259,"Flat Coil 0.2242"" Nom / 6.250"" / MS252 /",PUR-RAW,RUAN-RCVD,LB,1.0,0,OSP,132310


In [22]:
# Check data types
invBysubinventory_df.dtypes

Subinventory                 object
Subinventory Description     object
Part Number                  object
M/B                          object
Item Status                  object
Std Cost                    float64
dtype: object

In [23]:
# Check data types
Inventory_df.dtypes

Part Number       object
Description       object
Cost Category     object
Subinventory      object
UOM               object
Quantity         float64
Value              int64
Parameter         object
Account           object
dtype: object

In [24]:

# Strip leading and trailing spaces from column values. Merge will have NaN values if column values are not identical.
Inventory_df["Part Number"] = Inventory_df["Part Number"].str.strip()
invBysubinventory_df["Part Number"] = invBysubinventory_df["Part Number"].str.strip()

Inventory_df["Subinventory"] = Inventory_df["Subinventory"].str.strip()
invBysubinventory_df["Subinventory"] = invBysubinventory_df["Subinventory"].str.strip()

# Round dataframe numeric values to two decimal places to create matching column values in the two dataframes...also to avoid NaN values.
Inventory_df.round(decimals=3)
invBysubinventory_df.round(decimals=3)

Inventory_df['Quantity'] = np.round(Inventory_df['Quantity'], decimals = 3)
Inventory_df['Value'] = np.round(Inventory_df['Value'], decimals = 3)

# Join with Inventory by Subinventory datafarme.
inventory_df2 = pd.merge(Inventory_df,invBysubinventory_df, how="left", on=["Part Number","Subinventory"])

inventory_df2

,Part Number,Description,Cost Category,Subinventory,UOM,Quantity,Value,Parameter,Account,Subinventory Description,M/B,Item Status,Std Cost
0,1752699,"Flat Coil 0.1793"" Nom / 59.0"" / MS252 /",PUR-RAW,FAB,LB,1.0,2,Material,132101,Fab,Buy,Active,0.1
1,1764529,"Flat Coil 0.1793"" Nom / 52.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,2,Material,132101,Decoil - Steel Coils,Buy,Active,0.1
2,1764699,"Flat Coil 0.1793"" Nom / 59.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,2,Material,132101,Decoil - Steel Coils,Buy,Active,0.1
3,19524875,"Flat Coil 0.1046"" Nom / 4.875"" / MS264 /",PUR-RAW,FAB,LB,1.0,2,Material,132101,Fab,Buy,Active,0.1
4,22526259,"Flat Coil 0.2242"" Nom / 6.250"" / MS252 /",PUR-RAW,RUAN-RCVD,LB,1.0,2,Material,132101,Ruan RECEIVING DESCREPANCY,Buy,Active,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,5164689,"Flat Coil 0.3125"" Nom / 68.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,0,OSP,132310,Decoil - Steel Coils,Buy,Active,0.1
111,5764699,"Flat Coil 0.3750"" Nom / 59.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,0,OSP,132310,Decoil - Steel Coils,Buy,Active,0.1
112,5964699,"Flat Coil 0.5000"" Nom / 59.0"" / MS264",PUR-RAW,DECOIL,LB,1.0,0,OSP,132310,Decoil - Steel Coils,Buy,Active,0.1
113,22526259,"Flat Coil 0.2242"" Nom / 6.250"" / MS252 /",PUR-RAW,RUAN-RCVD,LB,1.0,0,OSP,132310,Ruan RECEIVING DESCREPANCY,Buy,Active,0.1


##Group data into new dataframes to mimic pivot tables
* By Account Number
* By Subinventory
* By Subinventory and Parameter
* By Make/Buy
* By Cost Category

In [25]:
# By Account Number

# Drop unnecessary columns
inventory_df3= inventory_df2.drop(columns=["Part Number","Description","Cost Category","UOM","Quantity","Parameter","Subinventory Description","M/B","Item Status","Std Cost"])

# Group by Account Number
by_acct_df= inventory_df3.groupby(by=['Account']).sum()

# Format as US currency
#by_acct_df['Value'] = by_acct_df['Value'].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))

by_acct_df


,Value
Account,
132101,46
132302,69
132303,0
132306,0
132310,0


In [26]:
# By Subinventory

# Drop unnecessary columns
inventory_df3= inventory_df2.drop(columns=["Part Number","Description","Cost Category","UOM","Quantity","Parameter","Account","M/B","Item Status","Std Cost"])

# Group by Account Number
by_subinventory_df= inventory_df3.groupby(by=['Subinventory']).sum()

# Format as US currency
#by_subinventory_df['Value'] = by_subinventory_df['Value'].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))

by_subinventory_df

,Value
Subinventory,
DECOIL,25
FAB,80
RUAN-RCVD,10


In [27]:
# By Make/Buy

# Drop unnecessary columns
inventory_df3= inventory_df2.drop(columns=["Part Number","Description","Cost Category","UOM","Quantity","Parameter","Account","Subinventory","Item Status","Std Cost"])

# Group by Account Number
by_MakeBuy_df= inventory_df3.groupby(by=['M/B']).sum()

# Format as US currency
#by_MakeBuy_df['Value'] = by_MakeBuy_df['Value'].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))

by_MakeBuy_df

,Value
M/B,
Buy,115


In [28]:
# By Cost Category

# Drop unnecessary columns
inventory_df3= inventory_df2.drop(columns=["Part Number","Description","M/B","UOM","Quantity","Parameter","Account","Subinventory","Item Status","Std Cost"])

# Group by Account Number
by_CostCategory_df= inventory_df3.groupby(by=['Cost Category']).sum()

# Format as US currency
#by_CostCategory_df['Value'] = by_CostCategory_df['Value'].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))

by_CostCategory_df

,Value
Cost Category,
PUR-RAW,115


In [29]:
# By Subinventory and Resource Type

# Drop unnecessary columns
elemental_inv_df3= elemental_inv_df.drop(columns=["Part Number","Description","UOM","Quantity","Category"])

# Rename columns to match previous dataframes.
elemental_inv_df3 = elemental_inv_df3.rename(columns={"Out Proc": 'OSP',"Mat'l OH":'Matl_OH'})

# Group by Account Number
by_Parameter_df= elemental_inv_df3.groupby(by=['Subinventory']).sum()

# Format as US currency
#by_Parameter_df['Material']= by_Parameter_df['Material'].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))
#by_Parameter_df['Matl_OH'] = by_Parameter_df['Matl_OH'].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))
#by_Parameter_df['Resource'] = by_Parameter_df['Resource'].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))
#by_Parameter_df['Overhead'] = by_Parameter_df['Overhead'].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))
#by_Parameter_df['OSP'] = by_Parameter_df['OSP'].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))

by_Parameter_df

,Material,Resource,Overhead,Matl_OH,OSP
Subinventory,,,,,
COIL 7,2,0,0,3,0
COIL 8,2,0,0,3,0
COIL 12,2,0,0,3,0
COIL 27,2,0,0,3,0
COIL 33,2,0,0,3,0
DECOIL,10,0,0,15,0
DECOIL 1,2,0,0,3,0
FAB,32,0,0,48,0
FAB 1,2,0,0,3,0


## Add the Subinventory Account listed in dataframe to the Accounts worksheet in Data.xlsx.  Data will be missing records if this is not completed.

### * Re-run the code with the new Data.xlsx file once the Account(s) have been added.

In [30]:
# Check for new subinventory accounts in the Elemental Inventory data that need to be added to the Accounts table. Left join.
# Any new accounts will appear underneath the heading "Subinventory_merge"

new_accounts_df= elem_inv_subs_df.merge(accounts_culled_df2.drop_duplicates(), on=['Subinventory'], 
                   how='left', indicator=True)

#new_accounts_df

new_accounts_df.query('_merge != "both"')

,Subinventory,_merge
1,COIL 8,left_only
3,COIL 33,left_only
4,FAB 1,left_only
5,COIL 12,left_only
6,DECOIL 1,left_only
7,COIL 27,left_only
8,COIL 7,left_only


## Save Inventory_df as Excel files for export.  

### Two options exist- 
###    1) Save as a standalone file named "Inventory.xlsx" or
###    2) Save a new sheet in the original "Data.xlsx" file

### In either case the file(s) will sit in the left pane inside the file icon and must be downloaded via the three dots to the right when file name is hovered over.


In [31]:
# Export the full Inventory table and the subset dataframes as new sheets in the existing "Data.xlsx" Excel file.
# The original reports remain. 
# File is in the left pane and will need downloading to your computer.
# This step takes 3-4 minutes...grab a Diet Mt Dew.

#with pd.ExcelWriter('Data.xlsx', mode='a') as writer:  

#    inventory_df2.to_excel(writer, sheet_name='Inventory')
#    by_acct_df.to_excel(writer, sheet_name='Account')
#    by_MakeBuy_df.to_excel(writer, sheet_name='MakeBuy')
#    by_CostCategory_df.to_excel(writer, sheet_name='Cost Category')
#    by_subinventory_df.to_excel(writer, sheet_name='Subinventory')
#    by_Parameter_df.to_excel(writer, sheet_name='SubInv & Parameter')

In [32]:
# Create variables for the number of rows in each dataframe
inventory_rows = len(inventory_df2.index)
by_acct_rows = len(by_acct_df.index)
by_MakeBuy_rows = len(by_MakeBuy_df.index)
by_CostCategory_rows = len(by_CostCategory_df.index)
by_subinventory_rows = len(by_subinventory_df.index)
by_Parameter_rows = len(by_Parameter_df.index)

In [33]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter("Inventory_Breakdown.xlsx", engine='xlsxwriter')

# Import dependencies
import pandas.io.formats.excel
pandas.io.formats.excel.ExcelFormatter.header_style = None
from xlsxwriter.utility import xl_rowcol_to_cell

# Convert the dataframe to an XlsxWriter Excel object. 
elemental_inv_data.to_excel(writer, sheet_name='Elemental_Inventory', startrow=1, header=False)
inv_bySubinv_data.to_excel(writer, sheet_name='Inv_bySubinv', startrow=1, header=False)
accounts.to_excel(writer, sheet_name='Accounts', startrow=1, header=False)

inventory_df2.to_excel(writer, sheet_name='Inventory', startrow=1, header=False)
by_acct_df.to_excel(writer, sheet_name='Account', startrow=1, header=False)
by_MakeBuy_df.to_excel(writer, sheet_name='MakeBuy', startrow=1, header=False)
by_CostCategory_df.to_excel(writer, sheet_name='Cost Category', startrow=1, header=False)
by_subinventory_df.to_excel(writer, sheet_name='Subinventory', startrow=1, header=False)
by_Parameter_df.to_excel(writer, sheet_name='SubInv & Parameter', startrow=1, header=False)

In [34]:
# Format exported Excel workbook sheets utilizing xlsxwriter

# Access workbook and worksheet(s)
workbook= writer.book

worksheet_EI= writer.sheets['Elemental_Inventory']
worksheet_subInv= writer.sheets['Inv_bySubinv']
worksheet_accounts= writer.sheets['Accounts']

worksheet_Inv= writer.sheets['Inventory']
worksheet_Acct= writer.sheets['Account']
worksheet_MB= writer.sheets['MakeBuy']
worksheet_CC= writer.sheets['Cost Category']
worksheet_Sub= writer.sheets['Subinventory']
worksheet_Par= writer.sheets['SubInv & Parameter']

# Resize worksheet
#worksheet.set_zoom(90)

# Format columns - setting variables
numerical_fmt = workbook.add_format()
numerical_fmt.set_num_format(4)

cell_format = workbook.add_format()
cell_format.set_align('center')

border_fmt = workbook.add_format()
border_fmt.set_bottom(0)

# Total formatting
total_fmt = workbook.add_format({
    'bold': True,
    'top': True,
    'text_wrap': True,
    'num_format': '#,##0.00',
    'valign': 'top',
    'align': 'center',
    'fg_color': '#FFD580'})

# Add an orange header format.
header_format = workbook.add_format({
    'bold': True,
    'bottom': True,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'fg_color': '#FFD580'})

# Add a gray header format.
header_format_gray = workbook.add_format({
    'bold': True,
    'bottom': True,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'fg_color': '#C5C6D0'})



In [35]:
# Format Elemental_Inventory Report worksheet

# Set column widths and numerical formats
worksheet_EI.set_column('A:A', 10, border_fmt)
worksheet_EI.set_column('B:B', 10, cell_format)
worksheet_EI.set_column('C:C', 18)
worksheet_EI.set_column('D:E', 18, cell_format)
worksheet_EI.set_column('F:F', 16, cell_format)
worksheet_EI.set_column('G:M', 12, numerical_fmt)

# Write the column headers with the defined format.
for col_num, value in enumerate(elemental_inv_data.columns.values):
    worksheet_EI.write(0, col_num + 1, value, header_format_gray)


In [36]:
# Format Inv_bySubinv Report worksheet

# Set column widths and numerical formats
worksheet_subInv.set_column('A:A', 8, border_fmt)
worksheet_subInv.set_column('B:B', 13.5, cell_format)
worksheet_subInv.set_column('C:C', 18, cell_format)
worksheet_subInv.set_column('D:D', 19.5)
worksheet_subInv.set_column('E:E', 15, cell_format)
worksheet_subInv.set_column('F:F', 18, cell_format)
worksheet_subInv.set_column('G:H', 14, cell_format)
worksheet_subInv.set_column('I:J', 14, numerical_fmt)
worksheet_subInv.set_column('K:L', 12, cell_format)
worksheet_subInv.set_column('M:M', 10, numerical_fmt)

# Write the column headers with the defined format.
for col_num, value in enumerate(inv_bySubinv_data.columns.values):
    worksheet_subInv.write(0, col_num + 1, value, header_format_gray)



In [37]:
# Format Accounts worksheet

# Set column widths and numerical formats
worksheet_accounts.set_column('A:A', 10, border_fmt)
worksheet_accounts.set_column('B:D', 17, cell_format)

# Write the column headers with the defined format.
for col_num, value in enumerate(accounts.columns.values):
    worksheet_accounts.write(0, col_num + 1, value, header_format_gray)

In [38]:
# Format Inventory worksheet

# Set column widths and numerical formats
worksheet_Inv.set_column('A:A', 8, border_fmt)
worksheet_Inv.set_column('B:B', 13.5, cell_format)
worksheet_Inv.set_column('C:C', 38, cell_format)
worksheet_Inv.set_column('D:D', 19.5)
worksheet_Inv.set_column('E:E', 15, cell_format)
worksheet_Inv.set_column('F:F', 8, cell_format)
worksheet_Inv.set_column('G:H', 15, numerical_fmt)
worksheet_Inv.set_column('I:I', 12, cell_format)
worksheet_Inv.set_column('J:J', 10, cell_format)
worksheet_Inv.set_column('K:K', 27.5)
worksheet_Inv.set_column('L:M', 8.45, cell_format)
worksheet_Inv.set_column('N:N', 10.5, numerical_fmt)

# Write the column headers with the defined format.
for col_num, value in enumerate(inventory_df2.columns.values):
    worksheet_Inv.write(0, col_num + 1, value, header_format)







In [39]:
# Format Account worksheet

# Set column widths and numerical formats
worksheet_Acct.set_column('A:A', 10, border_fmt)
worksheet_Acct.set_column('B:B', 16.75, numerical_fmt)

# Write the column headers with the defined format.
for col_num, value in enumerate(by_acct_df.columns.values):
    worksheet_Acct.write(0, col_num + 1, value, header_format)

# Add total rows
for column in range(0, 2):
    # Determine where we will place the formula
    cell_location = xl_rowcol_to_cell(by_acct_rows+1, column)
    # Get the range to use for the sum formula
    start_range = xl_rowcol_to_cell(1, column)
    end_range = xl_rowcol_to_cell(by_acct_rows, column)
    # Construct and write the formula
    formula = "=SUM({:s}:{:s})".format(start_range, end_range)
    worksheet_Acct.write_formula(cell_location, formula, total_fmt)

# Add a total label
worksheet_Acct.write_string(by_acct_rows+1, 0, "Total",total_fmt)

0

In [40]:
# Format Make/Buy worksheet

# Set column widths and numerical formats
worksheet_MB.set_column('A:A', 10, border_fmt)
worksheet_MB.set_column('B:B', 16.75, numerical_fmt)

# Write the column headers with the defined format.
for col_num, value in enumerate(by_MakeBuy_df.columns.values):
    worksheet_MB.write(0, col_num + 1, value, header_format)

# Add total rows
for column in range(0, 2):
    # Determine where we will place the formula
    cell_location = xl_rowcol_to_cell(by_MakeBuy_rows+1, column)
    # Get the range to use for the sum formula
    start_range = xl_rowcol_to_cell(1, column)
    end_range = xl_rowcol_to_cell(by_MakeBuy_rows, column)
    # Construct and write the formula
    formula = "=SUM({:s}:{:s})".format(start_range, end_range)
    worksheet_MB.write_formula(cell_location, formula, total_fmt)

# Add a total label
worksheet_MB.write_string(by_MakeBuy_rows+1, 0, "Total",total_fmt)

0

In [41]:
# Format Cost Category worksheet

# Set column widths and numerical formats
worksheet_CC.set_column('A:A', 21.3, border_fmt)
worksheet_CC.set_column('B:B', 16.75, numerical_fmt)

# Write the column headers with the defined format.
for col_num, value in enumerate(by_CostCategory_df.columns.values):
    worksheet_CC.write(0, col_num + 1, value, header_format)

# Add total rows  
for column in range(0, 2):
    # Determine where we will place the formula
    cell_location = xl_rowcol_to_cell(by_CostCategory_rows+1, column)
    # Get the range to use for the sum formula
    start_range = xl_rowcol_to_cell(1, column)
    end_range = xl_rowcol_to_cell(by_CostCategory_rows, column)
    # Construct and write the formula
    formula = "=SUM({:s}:{:s})".format(start_range, end_range)
    worksheet_CC.write_formula(cell_location, formula, total_fmt)

# Add a total label
worksheet_CC.write_string(by_CostCategory_rows+1, 0, "Total",total_fmt)

0

In [42]:
# Format Subinventory worksheet

# Set column widths and numerical formats
worksheet_Sub.set_column('A:A', 21.3, border_fmt)
worksheet_Sub.set_column('B:B', 16.75, numerical_fmt)

# Write the column headers with the defined format.
for col_num, value in enumerate(by_subinventory_df.columns.values):
    worksheet_Sub.write(0, col_num + 1, value, header_format)

# Add total rows  
for column in range(0, 2):
    # Determine where we will place the formula
    cell_location = xl_rowcol_to_cell(by_subinventory_rows+1, column)
    # Get the range to use for the sum formula
    start_range = xl_rowcol_to_cell(1, column)
    end_range = xl_rowcol_to_cell(by_subinventory_rows, column)
    # Construct and write the formula
    formula = "=SUM({:s}:{:s})".format(start_range, end_range)
    worksheet_Sub.write_formula(cell_location, formula, total_fmt)

# Add a total label
worksheet_Sub.write_string(by_subinventory_rows+1, 0, "Total",total_fmt)


0

In [43]:
# Format Subinventory and Parameters worksheet

# Set column widths and numerical formats
worksheet_Par.set_column('A:A', 21.3, border_fmt)
worksheet_Par.set_column('B:F', 16.75, numerical_fmt)

# Write the column headers with the defined format.
for col_num, value in enumerate(by_Parameter_df.columns.values):
    worksheet_Par.write(0, col_num + 1, value, header_format)

In [44]:
# Close the Pandas Excel writer and output the Excel file.
# It can take a minute for file to appear in left pane after code block has executed.
writer.save()